In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from ipynb.fs.full.Sports_NN import get_profit

In [4]:
def get_lm_model(feature_df):
    """
    Creates a basic OLS from the feature datafram
    Args:
        feature_df
    Returns:The total profit of this model for a given year, the percentage of games guessed correctly, as well as the mean and 
    standard deviation of the the difference between the actual and predicted score
    """    
    df = feature_df[['away_past_average_yards/play',
    'home_past_average_yards/play',
    'away_past_starting_yard_line',
    'home_past_starting_yard_line',
    'away_schedule_strength',
    'home_schedule_strength',
    'spread', 
    'margin',
    'id']]

    df = df[pd.to_numeric(df['home_past_average_yards/play'], errors='coerce').notnull()]
    df = df[pd.to_numeric(df['away_past_average_yards/play'], errors='coerce').notnull()]
    df = df[pd.to_numeric(df['home_past_starting_yard_line'], errors='coerce').notnull()]
    df = df[pd.to_numeric(df['away_past_starting_yard_line'], errors='coerce').notnull()]
    df = df[pd.to_numeric(df['home_schedule_strength'], errors='coerce').notnull()]
    df = df[pd.to_numeric(df['away_schedule_strength'], errors='coerce').notnull()]
    df = df[pd.to_numeric(df['spread'], errors='coerce').notnull()]


    labels = df.pop('margin')


    s_1 = df.pop('away_past_average_yards/play')
    s_2 = df.pop('home_past_average_yards/play')
    s_3 = df.pop('away_past_starting_yard_line')
    s_4 = df.pop('home_past_starting_yard_line')
    s_5 = df.pop('away_schedule_strength')
    s_6 = df.pop('home_schedule_strength')
    s_9 = df.pop('spread')*-1
    s_10 = df.pop('id')

    feature_df = pd.DataFrame()
    feature_df = pd.concat([s_1, s_2, s_3, s_4, s_5, s_6,s_9,s_10, labels], axis=1).reset_index()
    feature_df = feature_df.drop( 'index', axis = 1)
    feature_df = np.array(feature_df).astype('float32')

    np.random.shuffle(feature_df)

    training, test, = feature_df[:350,:], feature_df[350:,:]
    x_train = training[:,[0,1,2,3,4,5,6]]
    y_train = training[:,8]
    train_spread = training[:,[6]]
    train_id = training[:,7]

    x_test = test[:,[0,1,2,3,4,5,6]]
    y_test = test[:,8]
    test_spread = test[:,[6]]
    test_id = test[:,7]

    y_train = y_train * .01
    y_test = y_test * .01
    spread = test_spread
    shape = test_spread.shape

    linear_reg = LinearRegression().fit(x_train,y_train)
    prediction = linear_reg.predict(x_test)
    prediction  = np.reshape(prediction, (shape[0],1) )


    spread = np.reshape(spread, (shape[0],1))
    spread = spread*.01
    print()
    z = np.array([(y_test)])
    z = z.swapaxes(0,1)

    actual_scores = z

    actual_vs_pred = np.concatenate((z, prediction), axis = 1)
    profit_arr = np.concatenate((actual_vs_pred, spread), axis = 1)

    correct_win = (np.sign(actual_scores) == np.sign(prediction))
    difference = actual_scores-prediction
    print("% of correct winning team")
    print(correct_win.sum()/correct_win.size)
    win_percent = correct_win.sum()/correct_win.size
    print()
    print("Mean")
    mean = difference.mean()
    print(difference.mean())
    print()
    print("Standard Deviation")
    print(difference.std())
    sd = difference.std()
    print("")
    print("Total Profit")
    profit = get_profit(profit_arr)
    print(profit)
